In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob

In [2]:
os.getcwd()

'/home/jian/BigLots/2017_Daily_Sales_Weeks_Quarter_as_Transfer/2017_Q3'

In [17]:
file_list=glob.glob('/home/jian/BigLots/2017_Daily_Sales_Weeks_Quarter_as_Transfer/2017_Q3/*.txt')
file_list.remove('/home/jian/BigLots/2017_Daily_Sales_Weeks_Quarter_as_Transfer/2017_Q3/MediaStormDailySales01.txt')
next_week='/home/jian/BigLots/2017_Daily_Sales_Weeks_Quarter_as_Transfer/2017_Q4/MediaStormDailySales17Q4W120181204-052708-319.txt'
file_list.append(next_week)
file_list=sorted(file_list)

In [18]:
i=0
output=pd.DataFrame()
for file in file_list:
    print(datetime.datetime.now())
    df=pd.read_table(file,dtype=str,sep="|")
    df=df.drop_duplicates()
    df['subclass_transaction_amt']=df['subclass_transaction_amt'].astype(float)
    df['subclass_transaction_units']=df['subclass_transaction_units'].astype(int)
    df['rewards_label']=np.where(pd.isnull(df['customer_id_hashed']),"Non_Rewards","Rewards")
    df['transaction_dt']=df['transaction_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
    i+=1
    print("File "+str(i)+": "+str(df['transaction_dt'].min())+" to "+str(df['transaction_dt'].max()))
    
    df_sales=df.groupby(['location_id','rewards_label','transaction_dt'])['subclass_transaction_amt'].sum().to_frame().reset_index()
    df_sales=df_sales.rename(columns={"subclass_transaction_amt":"sales"})
    df_trans=df.groupby(['location_id','rewards_label','transaction_dt'])['subclass_transaction_amt'].count().to_frame().reset_index()
    df_trans=df_trans.rename(columns={"subclass_transaction_amt":"trans"})
    
    df=pd.merge(df_sales,df_trans,on=['location_id','rewards_label','transaction_dt'],how="left")
    output=output.append(df)

2018-12-05 11:17:21.122833
File 1: 2017-08-06 to 2017-08-12
2018-12-05 11:21:15.474948
File 2: 2017-08-13 to 2017-08-19
2018-12-05 11:25:21.863258
File 3: 2017-08-20 to 2017-08-26
2018-12-05 11:28:40.211263
File 4: 2017-08-27 to 2017-09-02
2018-12-05 11:32:30.085841
File 5: 2017-09-03 to 2017-09-09
2018-12-05 11:36:30.371058
File 6: 2017-09-10 to 2017-09-16
2018-12-05 11:39:43.064095
File 7: 2017-09-17 to 2017-09-23
2018-12-05 11:43:01.543659
File 8: 2017-09-24 to 2017-09-30
2018-12-05 11:47:32.583611
File 9: 2017-10-01 to 2017-10-07
2018-12-05 11:53:03.469156
File 10: 2017-10-08 to 2017-10-14
2018-12-05 11:56:20.405509
File 11: 2017-10-15 to 2017-10-21
2018-12-05 11:59:43.954313
File 12: 2017-10-22 to 2017-10-28
2018-12-05 12:03:15.851752
File 13: 2017-10-29 to 2017-11-04


In [25]:
output['location_id']=output['location_id'].astype(int)
def add_week_end_dt(x):
    if x.weekday()==6:
        y=x+datetime.timedelta(days=6)
    else:
        y=x+datetime.timedelta(days=5-x.weekday())
    return y
output['week_end_dt']=output['transaction_dt'].apply(lambda x: add_week_end_dt(x))
output=output.sort_values(['location_id','transaction_dt','rewards_label'])

In [28]:
output.to_csv("/home/jian/BigLots/2017_Daily_Sales_Weeks_Quarter_as_Transfer/2017_Q3/csv_2017_Q3_rewards_non_rewards.csv",index=False)